In [16]:
import pandas as pd
advertisers = pd.read_json('data/btertisers_dec1.json', lines=True)

In [17]:
buy_trades = pd.read_json('data/buy_trades_dec1.json', lines=True,)
sell_trades = pd.read_json('data/sell_trades_dec1.json', lines=True)

In [18]:
print(len(buy_trades), len(sell_trades), len(btertisers))

13052 45809 30164


In [19]:
bt = pd.concat(
    [pd.json_normalize(btertisers["userDetailVo"], sep="_", max_level=None),
    pd.json_normalize(btertisers["feedback"], sep="_", max_level=None)],
    axis=1)
bt["timestamp"] = btertisers["timestamp"]
bt.sort_values(by=["timestamp"], ascending=False, inplace=True)
bt.drop_duplicates(subset='userNo', keep='first', inplace=True); # unique Users

In [20]:
buy_trades.sort_values(by=["timestamp"], ascending=False, inplace=True)
buy_trades.drop_duplicates(subset=["btNo"], keep='first', inplace=True)
sell_trades.sort_values(by=["timestamp"], ascending=False, inplace=True)
sell_trades.drop_duplicates(subset=["btNo"], keep='first', inplace=True)

In [21]:
print(len(buy_trades), len(sell_trades), len(adv))

3492 6553 4522


In [25]:
bt = pd.merge(buy_trades, adv, how="left", left_on="advertiseruserNo", right_on="userNo")
st = pd.merge(sell_trades, adv, how="left", left_on="advertiseruserNo", right_on="userNo") 

In [26]:
print(len(bt), len(st))

3492 6553


In [30]:
bt["commissionRateByMaxPercent"] = bt["commissionRate"]*100/bt["maxSingleTransQuantity"]
bt["commissionRateByMinPercent"] = bt["commissionRate"]*100/bt["minSingleTransQuantity"]
st["commissionRateByMaxPercent"] = st["commissionRate"]*100/st["maxSingleTransQuantity"]
st["commissionRateByMinPercent"] = st["commissionRate"]*100/st["minSingleTransQuantity"]

In [32]:
pos_bins = [-0.01, 0.01, 0.1, 0.5, 0.75, 0.80, 0.85, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.00]
bin_labels = [0, 0.01, 0.1, 0.5, 0.75, 0.80, 0.85, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]

In [34]:
bt['pos_bin'] = pd.cut(bt['positiveRate'], pos_bins)
bt['neg_bin'] = pd.cut(bt['negativeRate'], pos_bins)
bt['pos_bin_label'] = pd.cut(bt['positiveRate'], pos_bins, labels=bin_labels)
bt['neg_bin_label'] = pd.cut(bt['negativeRate'], pos_bins, labels=bin_labels)

In [35]:
st['pos_bin'] = pd.cut(st['positiveRate'], pos_bins)
st['neg_bin'] = pd.cut(st['negativeRate'], pos_bins)
st['pos_bin_label'] = pd.cut(st['positiveRate'], pos_bins, labels=bin_labels)
st['neg_bin_label'] = pd.cut(st['negativeRate'], pos_bins, labels=bin_labels)

In [37]:
asset = st[["asset", "commissionRateByMaxPercent", "commissionRateByMaxPercent"]].\
    groupby(["asset"]).mean()
asset.to_csv("csv/commission_by_asset.csv")

In [40]:
pos = st[["pos_bin", "pos_bin_label", "commissionRateByMaxPercent", "commissionRateByMaxPercent"]].\
    groupby(["pos_bin"]).mean()
pos.to_csv("csv/commission_by_positive.csv")

neg = st[["neg_bin", "neg_bin_label", "commissionRateByMaxPercent", "commissionRateByMaxPercent"]].\
    groupby(["neg_bin"]).mean()
neg.to_csv("csv/commission_by_negative.csv")

/var/folders/s7/cbvrkwmd5vj_3ymczzxpsc940000gn/T/ipykernel_23776/2870190941.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pos = st[["pos_bin", "pos_bin_label", "commissionRateByMaxPercent", "commissionRateByMaxPercent"]].\
/var/folders/s7/cbvrkwmd5vj_3ymczzxpsc940000gn/T/ipykernel_23776/2870190941.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  neg = st[["neg_bin", "neg_bin_label", "commissionRateByMaxPercent", "commissionRateByMaxPercent"]].\
